In [12]:
import pandas as pd
import numpy as np
import re

In [144]:
# importer les datasets a merge
df_pays_anneeSouv=pd.read_csv("../datasets_scraped/df_pays_anneeSouv.csv")
df_pays_musees=pd.read_csv("../datasets_scraped/df_pays_musees.csv")
df_pays_nbPrix=pd.read_csv("../datasets_scraped/df_pays_nbPrix.csv")
df_pays_CategoriePrixNobel=pd.read_csv("../datasets_scraped/df_pays_CategoriePrixNobel.csv")
df_pays_univ=pd.read_csv("../datasets_scraped/df_pays_univ.csv")

## Etudier les differences entre les datasets des prix nobels

In [88]:
# Etudier les differences entre df_pays_CategoriePrixNobel et df_pays_nbPrix (les pays non référencés dans l'un ou l'autre des datasets)
df_pays_nbPrix2=df_pays_CategoriePrixNobel.groupby('Pays').count()
df_pays_nbPrix2.rename(columns={"CategoriePrixNobel":"nbPrix"}, inplace=True)
df_nbPrix_diff=df_pays_nbPrix2.join(df_pays_nbPrix.set_index('Pays'))

df_nbPrix_diff_nan=df_nbPrix_diff[df_nbPrix_diff['Nb_prixNobel'].isna()]
df_nbPrix_diff_nan

,nbPrix,Nb_prixNobel
Pays,,
Armenia,1,NaN
Brazil,1,NaN
China (People's Republic of China),8,NaN
"Congo, Democratic Republic",1,NaN
Ireland,11,NaN
Myanmar (Burma),1,NaN
Palestine,1,NaN
Russia and Soviet Union,30,NaN
Yugoslavia,3,NaN


In [89]:
df_nbPrix_diff2=df_pays_nbPrix.set_index('Pays').join(df_pays_nbPrix2)

df_nbPrix_diff2_nan=df_nbPrix_diff2[df_nbPrix_diff2['nbPrix'].isna()]
df_nbPrix_diff2_nan

,Nb_prixNobel,nbPrix
Pays,,
Russia,30,NaN
Republic of Ireland,11,NaN
China,8,NaN
Algeria,2,NaN
Democratic Republic of the Congo,1,NaN
Myanmar,1,NaN
State of Palestine,1,NaN
Socialist Federal Republic of Yugoslavia,3,NaN


In [90]:
df_pays_nbPrix2.reset_index(inplace=True)
df_pays_nbPrix2.rename(columns={"nbPrix":"Nb_prixNobel"}, inplace=True)
df_pays_nbPrix_final=pd.concat([df_pays_nbPrix,df_pays_nbPrix2[df_pays_nbPrix2.Pays.isin(['Armenia','Brazil'])]],ignore_index=True)
df_pays_nbPrix_final

,Pays,Nb_prixNobel
0,United States,411
1,United Kingdom,137
2,Germany,115
3,France,75
4,Sweden,34
...,...,...
78,Vietnam,1
79,Yemen,1
80,Socialist Federal Republic of Yugoslavia,3
81,Armenia,1


## Combiner les datasets relatifs à nbPrix et musée

### Identifier les pays en communs et traiter les mal étiquetés 

In [113]:
df_merged1=df_pays_musees.set_index('Pays').join(df_pays_nbPrix_final.set_index('Pays'))
df_merged1

,Notes_musees,NbVotes_musees_clean,Nb_prixNobel
Pays,,,
India,"['4.6', '4.5', '4.4', '4.6', '4.6', '4.3', '4....","[18000.0, 49000.0, 69000.0, 34000.0, 74000.0, ...",10.0
China,"['4.3', '4.4', '4.6', '4.5', '4.3', '4.4', '4....","[398.0, 925.0, 1000.0, 236.0, 154.0, 451.0, 38...",8.0
United States,"['4.3', '4.5', '4.8', '4.8', '4.8', '4.8', '4....","[373.0, 19000.0, 46000.0, 30000.0, 32000.0, 16...",411.0
Indonesia,"['4,7', '4,5', '4,5', '4,6', '4,3', '4,7', '4,...","[1000.0, 157000.0, 19.0, 22000.0, 205.0, 9000....",NaN
Pakistan,"['4.3', '4.5', '4.5', '4.6', '4.4', '4.4', '4....","[1000.0, 4000.0, 8000.0, 13000.0, 12000.0, 400...",2.0
...,...,...,...
Turks and Caicos Islands,"['4.4', '4.6', '5.0', '3.8', '4.6']","[187.0, 36.0, 4.0, 9.0, 7.0]",NaN
Marshall Islands,['4.2'],[34.0],NaN
Liechtenstein,"['4.6', '4.3', '3.7', '4.3', '4.2', '4.3', '4....","[280.0, 645.0, 64.0, 260.0, 17.0, 22.0, 18.0, ...",NaN


In [114]:
df_merged1_nan=df_merged1[df_merged1.Nb_prixNobel.isna()]
df_merged1_nan

,Notes_musees,NbVotes_musees_clean,Nb_prixNobel
Pays,,,
Indonesia,"['4,7', '4,5', '4,5', '4,6', '4,3', '4,7', '4,...","[1000.0, 157000.0, 19.0, 22000.0, 205.0, 9000....",NaN
Congo,"['4,1', '4,2', '4,4', '3,7', '4,3', '4,6']","[230.0, 1000.0, 9.0, 3.0, 87.0, 38.0]",NaN
Thailand,"['4.5', '4.3', '4.6', '4.5', '4.6', '4.6', '4....","[6000.0, 376.0, 397.0, 1000.0, 6000.0, 2000.0,...",NaN
Uganda,"['4.1', '4.6', '4.1', '4.7', '4.4']","[2000.0, 10.0, 921.0, 66.0, 38.0]",NaN
Sudan,"['4.3', '4.3', '4.4', '4.1', '4.2', '4.1', '4.0']","[106.0, 163.0, 16.0, 41.0, 270.0, 54.0, 3.0]",NaN
...,...,...,...
Turks and Caicos Islands,"['4.4', '4.6', '5.0', '3.8', '4.6']","[187.0, 36.0, 4.0, 9.0, 7.0]",NaN
Marshall Islands,['4.2'],[34.0],NaN
Liechtenstein,"['4.6', '4.3', '3.7', '4.3', '4.2', '4.3', '4....","[280.0, 645.0, 64.0, 260.0, 17.0, 22.0, 18.0, ...",NaN


In [115]:
def has_common_name(liste_mots1, liste_mots2):
    
    liste_mots_communs=set()

    # Vérifier si on a au moins un mot en commun
    for mot in liste_mots1:
        if mot in liste_mots2:
            liste_mots_communs.add(mot)
    return liste_mots_communs

df_merged1_nan.reset_index(inplace=True)
liste_pays1=list(df_merged1_nan.Pays.apply(lambda x : x.split()))
liste_pays1=[mot for sublist in liste_pays1 for mot in sublist]

liste_pays2=list(df_pays_nbPrix_final.Pays.apply(lambda x : x.split()))
liste_pays2=[mot for sublist in liste_pays2 for mot in sublist]

liste_mots_communs=has_common_name(liste_pays1,liste_pays2)
liste_mots_communs

{'Congo',
 'Ireland',
 'Islands',
 'Korea',
 'New',
 'North',
 'Palestine',
 'Republic',
 'Saint',
 'South',
 'United',
 'and',
 'of',
 'the'}

In [123]:
# Mettre a jour le nom des pays de sorte à ce qu'ils correspondent à ceux du dataset musée
dict_convertNomPays={"Democratic Republic of the Congo":"Congo", "Republic of Ireland": "Ireland","State of Palestine":'Palestine'}
df_pays_nbPrix_final.Pays=df_pays_nbPrix_final.Pays.apply(lambda x : dict_convertNomPays[x] if x in dict_convertNomPays else x)
df_pays_nbPrix_final[df_pays_nbPrix_final.Pays=='Congo']

,Pays,Nb_prixNobel
55,Congo,1


In [128]:
df_merged1=df_pays_musees.set_index('Pays').join(df_pays_nbPrix_final.set_index('Pays'))
print("Le nombre de pays sans prix Nobel : ",np.sum(df_merged1.Nb_prixNobel.isna()))
# Remplacer les NaN par des 0=nombre de prix nobel
df_merged1.Nb_prixNobel=df_merged1.Nb_prixNobel.fillna(0)
df_merged1

Le nombre de pays sans prix Nobel :  119


,Notes_musees,NbVotes_musees_clean,Nb_prixNobel
Pays,,,
India,"['4.6', '4.5', '4.4', '4.6', '4.6', '4.3', '4....","[18000.0, 49000.0, 69000.0, 34000.0, 74000.0, ...",10.0
China,"['4.3', '4.4', '4.6', '4.5', '4.3', '4.4', '4....","[398.0, 925.0, 1000.0, 236.0, 154.0, 451.0, 38...",8.0
United States,"['4.3', '4.5', '4.8', '4.8', '4.8', '4.8', '4....","[373.0, 19000.0, 46000.0, 30000.0, 32000.0, 16...",411.0
Indonesia,"['4,7', '4,5', '4,5', '4,6', '4,3', '4,7', '4,...","[1000.0, 157000.0, 19.0, 22000.0, 205.0, 9000....",0.0
Pakistan,"['4.3', '4.5', '4.5', '4.6', '4.4', '4.4', '4....","[1000.0, 4000.0, 8000.0, 13000.0, 12000.0, 400...",2.0
...,...,...,...
Turks and Caicos Islands,"['4.4', '4.6', '5.0', '3.8', '4.6']","[187.0, 36.0, 4.0, 9.0, 7.0]",0.0
Marshall Islands,['4.2'],[34.0],0.0
Liechtenstein,"['4.6', '4.3', '3.7', '4.3', '4.2', '4.3', '4....","[280.0, 645.0, 64.0, 260.0, 17.0, 22.0, 18.0, ...",0.0


In [152]:
df_musees_nbPrix=df_merged1.copy()

## Combiner les datasets anneeSouv et nbPrix_musées

In [156]:
df_merged2=df_musees_nbPrix.join(df_pays_anneeSouv.set_index('Pays'))
df_merged2

,Notes_musees,NbVotes_musees_clean,Nb_prixNobel,Annee_souverainete
Pays,,,,
India,"['4.6', '4.5', '4.4', '4.6', '4.6', '4.3', '4....","[18000.0, 49000.0, 69000.0, 34000.0, 74000.0, ...",10.0,1947.0
China,"['4.3', '4.4', '4.6', '4.5', '4.3', '4.4', '4....","[398.0, 925.0, 1000.0, 236.0, 154.0, 451.0, 38...",8.0,NaN
United States,"['4.3', '4.5', '4.8', '4.8', '4.8', '4.8', '4....","[373.0, 19000.0, 46000.0, 30000.0, 32000.0, 16...",411.0,NaN
Indonesia,"['4,7', '4,5', '4,5', '4,6', '4,3', '4,7', '4,...","[1000.0, 157000.0, 19.0, 22000.0, 205.0, 9000....",0.0,1945.0
Pakistan,"['4.3', '4.5', '4.5', '4.6', '4.4', '4.4', '4....","[1000.0, 4000.0, 8000.0, 13000.0, 12000.0, 400...",2.0,1947.0
...,...,...,...,...
Turks and Caicos Islands,"['4.4', '4.6', '5.0', '3.8', '4.6']","[187.0, 36.0, 4.0, 9.0, 7.0]",0.0,NaN
Marshall Islands,['4.2'],[34.0],0.0,1986.0
Liechtenstein,"['4.6', '4.3', '3.7', '4.3', '4.2', '4.3', '4....","[280.0, 645.0, 64.0, 260.0, 17.0, 22.0, 18.0, ...",0.0,1813.0


In [157]:
# Analyser le nombre de pays pour lesquels on a pas d'annee de souverainete
print("Le nombre de pays sans annee de souverainte : ",np.sum(df_merged2.Annee_souverainete.isna()))
df_merged2_nan=df_merged2[df_merged2.Annee_souverainete.isna()]
df_merged2_nan

Le nombre de pays sans annee de souverainte :  40


,Notes_musees,NbVotes_musees_clean,Nb_prixNobel,Annee_souverainete
Pays,,,,
China,"['4.3', '4.4', '4.6', '4.5', '4.3', '4.4', '4....","[398.0, 925.0, 1000.0, 236.0, 154.0, 451.0, 38...",8.0,NaN
United States,"['4.3', '4.5', '4.8', '4.8', '4.8', '4.8', '4....","[373.0, 19000.0, 46000.0, 30000.0, 32000.0, 16...",411.0,NaN
Congo,"['4,1', '4,2', '4,4', '3,7', '4,3', '4,6']","[230.0, 1000.0, 9.0, 3.0, 87.0, 38.0]",1.0,NaN
Vietnam,[],[],1.0,NaN
Turkey,[],[],2.0,NaN
Myanmar,"['4,2', '4,3', '4,6', '4,2', '4,4', '4,1', '4,...","[1000.0, 473.0, 175.0, 571.0, 11.0, 78.0, 15.0...",1.0,NaN
Ecuador,"['4,6', '4,7', '4,7', '4,6', '4,6', '4,8', '4,...","[1000.0, 2000.0, 2000.0, 1000.0, 638.0, 885.0,...",0.0,NaN
Netherlands,"['4,6', '4,3', '4,6', '4,5', '4,6', '4,5', '4,...","[90000.0, 5000.0, 83000.0, 10000.0, 17000.0, 6...",22.0,NaN
Benin,"['4,3', '3,6', '3,9', '3,9', '3,9', '4,2', '4,...","[427.0, 106.0, 1000.0, 332.0, 335.0, 359.0, 6....",0.0,NaN


In [158]:
# Identifier les mots en communs entre le dataset que nous voulons join et celui des pays sans correspondance (retrouver les pays mal etiquetés)
df_merged2_nan.reset_index(inplace=True)
liste_pays1=list(df_merged2_nan.Pays.apply(lambda x : x.split()))
liste_pays1=[mot for sublist in liste_pays1 for mot in sublist]

liste_pays2=list(df_pays_anneeSouv.Pays.apply(lambda x : x.split()))
liste_pays2=[mot for sublist in liste_pays2 for mot in sublist]

liste_mots_communs=has_common_name(liste_pays1,liste_pays2)
liste_mots_communs

{'China',
 'Congo',
 'Eswatini',
 'Islands',
 'Macedonia',
 'Netherlands',
 'New',
 'North',
 'Republic',
 'Salvador',
 'United',
 'Vietnam',
 'and',
 'of'}

In [159]:
# Mettre a jour le nom des pays de sorte à ce qu'ils correspondent à ceux du dataset nbPrix_musée
dict_convertNomPays={"People's Republic of China":"China", "Republic of Congo": "Congo","Democratic Republic of Congo":'Congo', "Vietnam Vietnam":"Vietnam",
                     "Eswatini / Swaziland":"Eswatini", "North Macedonia":"Macedonia", "The Netherlands": "Netherlands", "Salvador":"El Salvador"}
df_pays_anneeSouv.Pays=df_pays_anneeSouv.Pays.apply(lambda x : dict_convertNomPays[x] if x in dict_convertNomPays else x)
df_pays_anneeSouv[df_pays_anneeSouv.Pays=='Netherlands']

,Pays,Annee_souverainete
165,Netherlands,1945


In [160]:
df_merged2=df_musees_nbPrix.join(df_pays_anneeSouv.set_index('Pays'))
print("Le nombre de pays sans annee de souverainte : ",np.sum(df_merged2.Annee_souverainete.isna()))
df_nbPrix_musees_anneeSouv=df_merged2.copy()
df_nbPrix_musees_anneeSouv=df_nbPrix_musees_anneeSouv.reset_index()

Le nombre de pays sans annee de souverainte :  34


In [161]:
df_nbPrix_musees_anneeSouv.to_csv("../datasets_scraped/df_nbPrix_musees_anneeSouv.csv", index=False)

## Combiner les datasets univTop500 et nbPrix_musées_anneeSouv

### Aggreger le dataset des rankings d'universités

In [145]:
# Recuperer les pays avec leurs Rang nettoyé (sans "=" au debut de certains rangs)
for i in range(df_pays_univ.shape[0]):
    premier_car= df_pays_univ.iloc[i,2:3][0][0]
    if premier_car=="=":
        df_pays_univ.iloc[i,2:3]=int(df_pays_univ.iloc[i,2:3][0][1:])
    df_pays_univ.iloc[i,1]=df_pays_univ.iloc[i,1][1:] #Surppimer les espaces au debut des noms des pays

df_pays_univ=df_pays_univ.iloc[:,1:] 
df_pays_univ.Rang=df_pays_univ.Rang.astype(int)
df_pays_univ

,Pays,Rang,Score
0,United States,1,100.0
1,United Kingdom,2,99.2
2,United Kingdom,3,98.9
3,United States,4,98.3
4,United States,5,98.1
...,...,...,...
496,China (Mainland),494,23.4
497,United States,494,23.4
498,Spain,494,23.4
499,South Korea,498,23.2


In [155]:
df_pays_univ_rang=df_pays_univ.iloc[:,:2].copy()
df_pays_univ_rangMoy=df_pays_univ_rang.groupby('Pays').mean()
df_pays_univ_rangMoy=df_pays_univ_rangMoy.apply(lambda x : np.round(x))
df_pays_univ_rangMoy

,Rang
Pays,
Argentina,95.0
Australia,202.0
Austria,309.0
Belarus,387.0
Belgium,217.0
Brazil,274.0
Brunei,387.0
Canada,226.0
Chile,224.0


In [158]:
df_pays_nbUniv=df_pays_univ.iloc[:,:2].copy()
df_pays_nbUniv=df_pays_nbUniv.groupby('Pays').count()
df_pays_nbUniv=df_pays_nbUniv.rename(columns={"Rang":"Nb_univTop500"})
df_pays_nbUniv

,Nb_univTop500
Pays,
Argentina,1
Australia,27
Austria,5
Belarus,1
Belgium,7
Brazil,4
Brunei,1
Canada,19
Chile,3


In [160]:
df_pays_univTop500_agg=df_pays_nbUniv.join(df_pays_univ_rangMoy)
df_pays_univTop500_agg

,Nb_univTop500,Rang
Pays,,
Argentina,1,95.0
Australia,27,202.0
Austria,5,309.0
Belarus,1,387.0
Belgium,7,217.0
Brazil,4,274.0
Brunei,1,387.0
Canada,19,226.0
Chile,3,224.0


In [166]:
#df_pays_univTop500_agg.reset_index(inplace=True)
df_pays_univTop500_agg.to_csv("../datasets_scraped/df_pays_univTop500_agg.csv", index=False)

### Combiner les datasets

In [174]:
# Merge df_pays_univTop500_agg avec dataset 
df_pays_univTop500_agg=pd.read_csv("../datasets_scraped/df_pays_univTop500_agg.csv")
df_nbPrix_musees_anneeSouv=pd.read_csv("../datasets_scraped/df_nbPrix_musees_anneeSouv.csv")
df_nbPrix_musees_anneeSouv_univTop500= df_nbPrix_musees_anneeSouv.set_index('Pays').join(df_pays_univTop500_agg.set_index('Pays'))
df_nbPrix_musees_anneeSouv_univTop500.Nb_univTop500=df_nbPrix_musees_anneeSouv_univTop500.Nb_univTop500.fillna(0)
df_nbPrix_musees_anneeSouv_univTop500

,Notes_musees,NbVotes_musees_clean,Nb_prixNobel,Annee_souverainete,Nb_univTop500,Rang
Pays,,,,,,
Afghanistan,[4.5],[574.0],0.0,1919.0,0.0,NaN
Albania,"['4,1', '4,4', '4,6', '4,6', '4,4', '4,7', '4,...","[1000.0, 1000.0, 471.0, 1000.0, 3000.0, 259.0,...",0.0,1912.0,0.0,NaN
Algeria,"['4,4', '4,6', '4,4', '4,3', '4,7', '4,6', '4,...","[250.0, 545.0, 790.0, 199.0, 170.0, 348.0, 240...",2.0,1962.0,0.0,NaN
Andorra,"['4,5', '4,5', '4,6', '4,4', '4,5', '4,7', '4,...","[383.0, 563.0, 153.0, 421.0, 193.0, 56.0, 128....",0.0,NaN,0.0,NaN
Angola,"['4,8', '4,2', '4,1', '3,9', '4,3', '4,5', '4,...","[4.0, 421.0, 191.0, 116.0, 423.0, 11.0, 261.0,...",0.0,1975.0,0.0,NaN
...,...,...,...,...,...,...
Venezuela,"['4,2', '4,5', '4,4', '4,3', '4,7', '4,2', '4,...","[620.0, 266.0, 243.0, 805.0, 13.0, 314.0, 699....",1.0,1811.0,0.0,NaN
Vietnam,[],[],1.0,1945.0,0.0,NaN
Yemen,"['4.4', '4.4', '4.2', '5.0', '5.0', '4.8', '4....","[244.0, 222.0, 147.0, 2.0, 6.0, 5.0, 7.0, 4.0,...",1.0,1990.0,0.0,NaN


In [175]:
df_nbPrix_musees_anneeSouv_univTop500.reset_index(inplace=True)
df_nbPrix_musees_anneeSouv_univTop500.to_csv("../datasets_scraped/df_nbPrix_musees_anneeSouv_univTop500.csv", index=False)